In [11]:
import secrets  

In [12]:
p = 97  
a = 2   
b = 3   


G = (3, 6)


O = None

In [13]:
def is_on_curve(x, y):
    
    return (y ** 2) % p == (x ** 3 + a * x + b) % p

In [14]:
def inverse_mod(k, p):
    
    if k == 0:
        raise ZeroDivisionError("Division by zero")
    return pow(k, p - 2, p)  

In [15]:
def point_add(P, Q):
    
    if P == O:
        return Q
    if Q == O:
        return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2 and y1 != y2:
        return O  

    if P == Q:  
        m = (3 * x1 * x1 + a) * inverse_mod(2 * y1, p)
    else:  
        m = (y2 - y1) * inverse_mod(x2 - x1, p)

    m %= p
    x3 = (m * m - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p

    return (x3, y3)

In [16]:
def scalar_mult(k, P):
    
    result = O
    while k > 0:
        if k % 2 == 1:
            result = point_add(result, P)
        P = point_add(P, P)
        k = k // 2
    return result

In [17]:
def generate_keypair():
    
    private_key = secrets.randbelow(p - 1) + 1
    public_key = scalar_mult(private_key, G)
    return private_key, public_key

In [18]:
def encrypt(Pm, public_key):
    
    k = secrets.randbelow(p - 1) + 1
    C1 = scalar_mult(k, G)
    S = scalar_mult(k, public_key)
    C2 = point_add(Pm, S)
    return (C1, C2)

In [19]:
def decrypt(C1, C2, private_key):
    
    S = scalar_mult(private_key, C1)
    S_inv = (S[0], (-S[1]) % p)
    Pm = point_add(C2, S_inv)
    return Pm

In [20]:
Pm = (3, 6)  

private_key, public_key = generate_keypair()
print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")

C1, C2 = encrypt(Pm, public_key)
print(f"Encrypted: C1 = {C1}, C2 = {C2}")

decrypted = decrypt(C1, C2, private_key)
print(f"Decrypted: {decrypted}")

Private Key: 26
Public Key: (3, 6)
Encrypted: C1 = (3, 6), C2 = (80, 10)
Decrypted: (3, 6)
